# Chapter 5. 파이썬의 함수는 일급 객체다.
- 파이썬에서 모든 함수는 일급이다.

## 5.1 함수를 객체처럼 다루기

In [21]:
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n * factorial(n-1)

In [22]:
print(factorial(42))
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
returns n!
<class 'function'>


In [23]:
fact = factorial
print(fact)

<function factorial at 0x10b65c820>


In [24]:
fact(5)

120

In [25]:
map(factorial, range(11))

In [26]:
list(map(fact, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## 5.2 고위 함수
> 함수를 인수로 받거나, 함수를 결과로 반환하는 함수를 고위 함수(higher-order function)라고 한다. 대표적으로 `map()`함수가 있다.  `list.sort()`와 `sorted()` 내장함수에서 소개한 `sorted()` 내장함수도 일급 함수의 예다.
예를 들어 다음의 예제처럼 길이에 따라 단어 리스트를 정렬하려면 len 함수를 key 인수로 전달하면 된다.

In [27]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [28]:
# 인수 하나를 받는 함수는 모두 key 인수로 사용할 수 있다.
def reverse(word):
    return word[::-1]

reverse('testing')

'gnitset'

In [29]:
# key 인수에 reverse 함수를 넘겨주면, reverse 함수가 반환하는 값으로 정렬한다.
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### 5.2.1 map(), filter(), reduce()의 대안

In [30]:
print(list(map(fact, range(6))))        # 0!에서 5!
print([fact(n) for n in range(6)])      # 동일한 연산을 컴프리헨션으로
print(list(map(factorial, filter(lambda n: n % 2, range(6)))))      # map과 filter를 사용해 0!에서 5!까지의 홀수 팩토리얼을 계산
print([factorial(n) for n in range(6) if n % 2])                    # 동일한 연산을 컴프리헨션으로


[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]
[1, 6, 120]
[1, 6, 120]


파이썬 3에서 map()과 filter()는 제너레이터를 반환하므로, 제너레이터 표현식이 이 함수들을 직접 대체한다.

In [31]:
from functools import reduce
from operator import add

print(reduce(add, range(100)))          # 0부터 99까지의 합
print(sum(range(100)))                  # 동일한 연산을 내장 함수로

4950
4950


고위 함수를 사용할 때 작은 일회용 함수를 생성하는 것이 편리할 때도 있다. 그렇기 때문에 익명 함수가 유용하게 사용된다. 익명 함수에 대해 알아보자.

## 5.3 익명 함수
`lambda`는 파이썬 표현식 내에 익명 함수를 생성한다. 그렇지만 파이썬의 단순한 구문이 람다 함수의 본체가 순수한 표현식으로만 구성되도록 제한한다. 즉, 람다에서는 할당문이나 while, try 등의 복합 문을 사용할 수 없다. 람다 함수는 이름이 없기 때문에, 람다 함수를 호출할 때는 람다 함수를 할당한 변수를 사용할 수 없다. 람다 함수는 일회용 함수로 사용하고, 람다 함수를 호출할 때는 람다 함수를 정의한 표현식을 사용한다.

In [32]:
# lambda를 사용해서 철자 역순으로 단어 리스트 정렬
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
print(sorted(fruits, key=lambda word: word[::-1]))

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']


## 5.4 일곱 가지 맛의 콜러블 객체 (??)
<!-- ㅋㅋㅋㅋㅋㅋㅋ -->
콜러블 객체는 호출 연산자 `()`를 사용할 수 있는 객체를 말한다. 파이썬에서는 함수, 메서드, 클래스, 클래스의 인스턴스, 제너레이터 함수, 제너레이터, 코루틴, 코루틴 데코레이터 등이 콜러블 객체이다.  
파이썬에서는 다양한 콜러블형이 존재하므로, callable() 내장 함수를 사용하여 객체가 콜러블 객체인지 확인하는 것이 가장 안전하다.

In [33]:
abs, str, 13

(<function abs(x, /)>, str, 13)

In [34]:
callable(abs), callable(str), callable(13)

(True, True, False)

## 5.5 사용자 정의 콜러블형
파이썬 함수가 실체 객체일 뿐만 아니라, 모든 파이썬 객체가 함수처럼 동작하게 만들 수 있다. 단지 `__call__()` 메서드만 구현하면 된다.

In [35]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        return self.pick()



In [36]:
bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())
print(callable(bingo))

0
2
True


## 5.6 함수 인트로스펙션
함수 객체는 __doc__ 이외에도 많은 속성을 가지고 있다. dir() 함수가 factorial() 함수의 속성을 보여준다.

In [37]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [47]:
def upper_case_name(obj):
    return ("%s %s" % (obj.first_name, obj.last_name)).upper()

upper_case_name.short_description = 'Customer name'
upper_case_name.short_description   # 이런 식으로 함수의 속성을 추가할 수 있다.

'Customer name'

In [48]:
# 일반함수의 고유한 속성을 알아보려면 두 집합의 차집합을 구하면 된다.

class C: pass
obj = C()
def func(): pass

sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

## 5.7 위치 매개변수에서 키워드 전용 매개변수까지
키워드 전용 인수(keyword-only argument)를 이용해서 향상된, 파이썬 3의지극히 융통성 있는 매개변수 처리 매커니즘은 파이썬 함수에서 볼 수 있는 가장 훌륭한 기능 중 하나다. 함수를 호출 할 때 반복 가능 객체나 매핑형을 별도의 인수로 '폭발'시키는 *와 ** 기호도 이 메커니즘과 밀접하게 연관되어 있다. 
<!-- 어떻게? -->

In [55]:
def tag(name, *connect, cls=None, **attr):
    """하나 이상의 HTML 태그를 생성한다."""
    if cls is not None:
        attr['class'] = cls
    if attr:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attr.items()))
    else:
        attr_str = ''
    if connect:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in connect)
    else:
        return '<%s%s />' % (name, attr_str)

In [50]:
tag('br')

'<br />'

In [51]:
tag('p', 'hello')

'<p>hello</p>'

In [57]:
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))
print(tag('p', 'hello', 'world', cls='sidebar'))
print(tag(content='testing', name="img"))

<p>hello</p>
<p>world</p>
<p id="33">hello</p>
<p class="sidebar">hello</p>
<p class="sidebar">world</p>
<img content="testing" />


In [56]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
          'src': 'sunset.jpg', 'cls': 'framed'}
print(tag(**my_tag))

<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />


## 5.8 매개변수에 대한 정보 읽기
이건 넘어갈게요. what is bobo

In [58]:
# 원하는 길이 가까이에 있는 공백에서 잘라서 문자열을 단축하는 함수
def clip(text, max_len=80):
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다."""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:     # 공백이 없다면 max_len까지만 잘라낸다.
        end = max_len
    return text[:end].rstrip()

In [61]:
print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames)
print(clip.__code__.co_argcount)

(80,)
<code object clip at 0x10516c9d0, file "/var/folders/3t/3nx07css1dq7_gssfk5d_rn00000gn/T/ipykernel_51404/2236947758.py", line 2>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


In [67]:
# 해당 모듈을 사용하면 더 깔끔하게 확인할 수 있다.
from inspect import signature

sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [68]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [77]:
sig = signature(tag)
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
            'src': 'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attr={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [75]:
for name, value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attr = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [78]:
del my_tag['name']
bound_args = sig.bind(**my_tag)

TypeError: missing a required argument: 'name'

## 5.9 함수 어노테이션
함수에 type hint를 달아서 주석을 추가할 수 있다는 뜻이다.

In [79]:
def clip(text: str, max_len: 'int > 0'=80) -> str:
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다."""
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:     # 공백이 없다면 max_len까지만 잘라낸다.
        end = max_len
    return text[:end].rstrip()

In [80]:
clip.__annotations__    # __annotations__ 속성에 함수의 매개변수와 반환값에 대한 주석이 저장된다.

{'text': str, 'max_len': 'int > 0', 'return': str}

## 5.10 함수형 프로그래밍을 우한 패키지
귀도 반 로섬씨는 파이썬이 함수형 프로그래밍 언어를 지향하지 않았다고 공표하고 있지만, operatordhk functools 같은 패키지의 지원으로 파이썬에서도 제법 함수형 코딩을 사용할 수 있다.

### 5.10.1 operator 모듈

In [81]:
from functools import reduce

def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

lambda a, b: a*b 같이 익명 함수를 작성하는 수고를 덜기 위해 operator는 수십 개의 연산자에 대응하는 함수를 제공한다.
<!-- (하지만 왜 이렇게 써야하는 지 이해는 안 된다. 어쨌든 그런 방법이 있다고 합니다.) -->

In [82]:
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))